In [4]:
import requests
import datetime as dt # 時間套件
import pandas as pd
from dateutil.relativedelta import relativedelta

def Get_N_Month_Data(month_num:int,stock_id:int) ->pd.DataFrame:
# 當日時間
    date_now = dt.datetime.now()

    # 建立日期串列
    date_list = [(date_now - relativedelta(months=i)).replace(day=1).\
                strftime('%Y%m%d') for i in range(month_num)]
    date_list.reverse()

    # 用於存儲每個月的數據
    all_data = []

    # 使用迴圈抓取連續月份資料
    for date in date_list:
        url = f'https://www.twse.com.tw/rwd/zh/afterTrading/\
            STOCK_DAY?date={date}&stockNo={stock_id}'
        try:
            json_data = requests.get(url).json()

            df = pd.DataFrame(data=json_data['data'],
                            columns=json_data['fields'])
        
            all_data.append(df)

        except Exception as e:
            print(f"無法取得{date}的資料, 可能資料量不足.")

    # 合併所有月份的數據
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    return final_df
    
def Get_Data_Dict(data:pd.DataFrame)->dict:
    
    try:
        if not data.empty: 
            columns_list = data.columns.tolist()
            datas_list = data.values.tolist()

            final_dict_list = []
            for row in datas_list:
                row_dict = {columns_list[i]: row[i] for i in range(len(columns_list))}
                final_dict_list.append(row_dict)
        
            return final_dict_list
        else:
            print("資料遺失或空白 DataFrame")
            return {}
    except Exception as e:
        print(f"發生錯誤: {str(e)}")
        return {}
    

    


In [5]:
from pydantic import BaseModel,RootModel,Field
from datetime import datetime

class StockData(BaseModel):
    date:str=Field(alias="日期")
    trading_volume:str=Field(alias="成交股數")
    turnover:str=Field(alias="成交金額")
    open_price:float=Field(alias="開盤價")
    high_price:float=Field(alias="最高價")
    low_price:float=Field(alias="最低價")
    close_price:float=Field(alias="收盤價")
    change:float=Field(alias="漲跌價差")
    transactions:str=Field(alias="成交筆數")

class Data(RootModel):
    root:list[StockData]


In [41]:
## rsi
def calculate_rsi(data:pd.DataFrame, window=14)->pd.Series:
    """
    Calculate RSI (Relative Strength Index) for given data.
    
    Parameters:
    - data: pandas DataFrame with 'Close' prices.
    - window: RSI window period (default is 14).
    
    Returns:
    - pandas Series with RSI values.
    """

    # data['收盤價'] = data['收盤價'].astype(float)

    # # 創建 Series 並計算差分
    # s = pd.Series(data['收盤價'])
    # print(s.diff())
    

    delta = pd.Series(data['收盤價'].astype(float).values).diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


In [52]:
import pandas as pd

def calculate_bollinger_bands(data: pd.DataFrame, window=20, num_std=2):
    # 計算移動平均線
    data['MA'] = data['收盤價'].rolling(window=window).mean()
    
    # 計算標準差
    data['std_dev'] = data['收盤價'].rolling(window=window).std()
    
    # 計算布林通道的上限線和下限線
    data['UpperBand'] = data['MA'] + num_std * data['std_dev']
    data['LowerBand'] = data['MA'] - num_std * data['std_dev']
    
    return data



In [57]:

def main():
    
    month_num=6
    stock_id=1101
    month_datas:pd.DataFrame=Get_N_Month_Data(month_num=month_num,stock_id=stock_id)
    
    data_list:list=Get_Data_Dict(month_datas)
    data:Data=Data.model_validate(data_list)
    stock_datas:list[dict]=data.model_dump()
    
    
    
    # 計算 RSI
    rsi:pd.Series = calculate_rsi(month_datas)
    month_datas['rsi']=rsi

    # print(month_datas)



    month_datas = calculate_bollinger_bands(month_datas)
    print(month_datas)

if __name__ =="__main__":
    main()

            日期        成交股數           成交金額    開盤價    最高價    最低價    收盤價   漲跌價差  \
0    113/01/02  14,937,049    518,751,792  34.85  34.85  34.60  34.70  -0.15   
1    113/01/03  12,793,849    438,994,226  34.50  34.75  34.15  34.20  -0.50   
2    113/01/04   6,473,946    221,350,197  34.20  34.30  34.15  34.20   0.00   
3    113/01/05   4,691,751    160,866,951  34.20  34.35  34.20  34.35  +0.15   
4    113/01/08   5,581,614    191,844,472  34.40  34.55  34.25  34.25  -0.10   
..         ...         ...            ...    ...    ...    ...    ...    ...   
107  113/06/17  13,657,934    463,303,134  34.00  34.15  33.80  34.00   0.00   
108  113/06/18  10,652,577    360,223,401  34.00  34.05  33.70  33.90  -0.10   
109  113/06/19  18,212,141    619,087,320  33.90  34.10  33.80  34.05  +0.15   
110  113/06/20  12,859,180    438,336,654  34.05  34.15  34.00  34.15  +0.10   
111  113/06/21  34,661,381  1,196,685,878  34.15  34.70  34.10  34.65  +0.50   

       成交筆數        rsi       MA   std_d